In [356]:
class Graph:
    def __init__(self):
        self.adj_dict = {}
        self.node_dict = {}
        self.edge_list = []
        
    def add_edge(self, v, u, pattern):
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:pattern}
        else:
            self.adj_dict[v][u] = pattern
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:pattern}
        else:
            self.adj_dict[u][v] = pattern
            
        if v not in self.node_dict:
            self.node_dict[v] = v
            
        if u not in self.node_dict:
            self.node_dict[u] = u
        
        self.edge_list.append([v,u])
        
    def del_edge(self, v, u):
        if u in self.adj_dict[v]:
            del(self.adj_dict[v][u])
            
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
    def del_uedge(self, v, u):
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
            
    def print_edge(self):
        return self.edge_list
        
    def print_adj(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = len(self.adj_dict[v])
        self.vertex_num = 1
        
        return self.vertex_num, self.deg, self.adj_list

In [357]:
import sys
import time

edges_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.edges' #sys.argv[1]
label_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.node_labels' #sys.argv[2]
# k = 7 #sys.argv[3]
label_dict = {}
graph_sort = []
label_kind = []
node_dict = {}
pt_fq = {} # 빈발패턴
label_f = open(label_path, 'r')
while True:
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        label_kind.append('0')
        break
    
    node = line.split(',')[0]
    label = line.split(',')[1]
    label_dict[node] = label
    label_kind.append(label)
label_f.close

label_kind = list(set(label_kind))

edge_f = open(edges_path, 'r')
gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    v = line.split(',')[0]
    u = line.split(',')[1]
    if v not in label_dict:
        label_dict[v] = '0'
        
    if u not in label_dict:
        label_dict[u] = '0'
    graph_sort.append([v,u])
    
    if v not in node_dict:
        node_dict[v] = v
    if u not in node_dict:
        node_dict[u] = u
        
    pt = label_dict[v] + '/' + label_dict[u]
    re_pt = ''.join(reversed(pt))
    
    if pt not in pt_fq and re_pt not in pt_fq:
        pt_fq[pt] = 1
        gs.add_edge(v,u,pt)
        
    elif re_pt in pt_fq:
        pt_fq[re_pt] = pt_fq[re_pt] + 1
        gs.add_edge(v,u,re_pt)
        
    elif pt in pt_fq:
        pt_fq[pt] = pt_fq[pt] + 1
        gs.add_edge(v,u,pt)

edge_f.close

new_pt_fq = {}
new_pt_fq.update(pt_fq)

In [358]:
p_id = 0
Freq = 0
TF = 0
FR = 0

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100 - 1
    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def pattern_gen(ptternFrequency, pttern_th):
    add_pt = {}
    
    for pt1 in ptternFrequency:
        if ptternFrequency[pt1] >= pttern_th:
            pt_tgt = pt1.split('/')
            for pt2 in ptternFrequency:
                if pt1 != pt2 and ptternFrequency[pt2] >= pttern_th:
                    pt_sce = pt2.split('/')
                    between_len = len(pt_tgt) - 2
                    if between_len == 0:
                        if pt_tgt[0] == pt_sce[0]:
                            pt = pt_sce[1] + '/' + pt1
                        elif pt_tgt[0] == pt_sce[1]:
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[-1] == pt_sce[0]:
                            pt = pt1 + '/' + pt_sce[1]

                        elif pt_tgt[-1] == pt_sce[-1]:
                            pt = pt1 + '/' + pt_sce[0]
                        else:
                            continue
                    else:
                        if pt_tgt[:between_len] == pt_sce[between_len:-1]: #앞, 뒤
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[between_len:-1] == pt_sce[:between_len]: #뒤, 앞
                            pt = pt1 + '/' + pt_sce[-1]
                        else:
                            continue
                        
                    re_pt = ''.join(reversed(pt))
        
                    if pt not in add_pt and re_pt not in add_pt:
                        add_pt[pt] = min(ptternFrequency[pt1], ptternFrequency[pt2])
    
    return add_pt

pers = 10
def updatePattern(updateDict, pers):
    update_dict = (pattern_gen(updateDict, Top_nPercent(updateDict, pers)))
    if len(update_dict) != 0:
        new_pt_fq.update(update_dict)
        updatePattern(update_dict, pers)
    
updatePattern(pt_fq, pers)
print(len(pt_fq))
print(len(new_pt_fq))
    
# print('a')
# a = pattern_gen(pt_fq, Top_nPercent(pt_fq, pers))
# print(len(a), Top_nPercent(pt_fq, pers))
# print('b')
# b = pattern_gen(a, Top_nPercent(a, pers))
# print(len(b), Top_nPercent(a, pers))
# print('c')
# c = pattern_gen(b, Top_nPercent(b, pers))
# print(len(c), Top_nPercent(b, pers))

# print(sorted(b.items(), key = lambda item: item[1], reverse = True))
            

1814
6947


In [359]:
# print(list(sorted(node_dict.items(), key = lambda item: item[1], reverse = True))[:10])
# print(list(sorted(node_dict.items(), key = lambda item: item[1]))[:10])
# print(list(sorted(new_pt_fq.items(), key = lambda item: item[1], reverse = True))[:10])
# print(list(sorted(new_pt_fq.items(), key = lambda item: item[1]))[:10])

In [360]:
def pattern_Marge(v, pt):
    score = {}
    
    for u in pt:
        re_pt = ''.join(reversed(pt[u]))
        if pt[u] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(v)[1]
            same_deg = len((pt.keys() & gs.vertex(u)[2].keys()))
            deg_ratio = gs.vertex(u)[1]/gs.vertex(v)[1]
            
            score[u] = [same_deg/deg_max + deg_ratio, pt[u]]
    
        gs.del_uedge(v, u)
    if score:
        su = list(sorted(score.items(), key = lambda item: item[1], reverse = True))[0]
        gs.del_edge(v, su[0])
        MargeNode = v + '/' + su[0]
        return MargeNode, su[1][1]
    else:
        return 0, 0
    
def pattern_Marge2(MargeNode, pt):
    frontV = MargeNode.split('/')[0]
    backV = MargeNode.split('/')[-1]
    union_list1 = {}
    union_list2 = {}
    frontScore = {}
    backScore = {}

    for front in gs.vertex(frontV)[2].keys():
        if backV ==  front:
            continue
        union_list1[front] = label_dict[front] + '/' + pt
        
    for back in gs.vertex(backV)[2].keys():
        if frontV == back:
            continue
        union_list2[back] = pt + '/' + label_dict[back]
            
    for union in union_list1.keys():
        re_pt = ''.join(reversed(union_list1[union]))
        
        if union_list1[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(frontV)[1]
            same_deg = len((gs.vertex(frontV)[2].keys() & gs.vertex(union)[2].keys()))
            deg_ratio = gs.vertex(union)[1]/gs.vertex(frontV)[1]
            
            frontScore[union] = same_deg/deg_max + deg_ratio
        
        gs.del_uedge(frontV, union)
            
    for union in union_list2.keys():
        re_pt = ''.join(reversed(union_list2[union]))
        if union_list2[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(backV)[1]
            same_deg = len((gs.vertex(backV)[2].keys() & gs.vertex(union)[2].keys()))
            deg_ratio = gs.vertex(union)[1]/gs.vertex(backV)[1]
            
            backScore[union] = same_deg/deg_max + deg_ratio
            
        gs.del_uedge(backV, union)

    if frontScore or backScore:
        if frontScore:
            frontMax = list(sorted(frontScore.items(), key = lambda item: item[1], reverse = True))[0]
        else:
            frontMax = [0,0]
            
        if backScore:
            backMax = list(sorted(backScore.items(), key = lambda item: item[1], reverse = True))[0]
        else:
            backMax = [0,0]
            
        if frontMax[1] >= backMax[1] and frontMax[1] != 0:
            gs.del_edge(frontV, frontMax[0])
            MargeNode = frontMax[0] + '/' + MargeNode
            new_pt = union_list1[frontMax[0]]
            
        elif frontMax[1] < backMax[1] and backMax[1] != 0:
            gs.del_edge(backV, backMax[0])
            MargeNode = MargeNode + '/' + backMax[0]
            new_pt = union_list2[backMax[0]]
            
        else:
            return MargeNode, pt, 0
            
        return MargeNode, new_pt, 1
    
    else:
        return MargeNode, pt, 0


In [374]:
for n in node_dict:
    node_dict[n] = gs.vertex(n)[1]
    
nodeList = []
node_list = sorted(node_dict.items(), key = lambda item: item[1], reverse = True)
for n in node_list:
    nodeList.append(n[0])


In [375]:
pattern_dict = {}
print(len(nodeList))
ZERONode = []
edgeList = []
while(True):
    if node_size <= 0: #48
        break
    
    node = nodeList[0]
    print(node)
    s1 = time.time()
    newV, mP = pattern_Marge(node, gs.vertex(node)[2])
    
    if newV == 0:
        nodeList.remove(node)
        ZERONode.append(node)
        print(f'ZERONode: {node}')
        continue
    
    boolz = 1
    while (boolz > 0):
        newV, mP, boolz = pattern_Marge2(newV, mP)
    
    del_list = newV.split('/')
    for del_one in del_list:
        nodeList.remove(del_one)
        edgeList.extend(list(gs.vertex(del_one)[2].keys()))
    
    if mP not in pattern_dict:
        pattern_dict[mP] = [{newV:edgeList}]
    else:
        pattern_dict[mP].append([{newV:edgeList}])

    node_size = len(nodeList)
    e1 = time.time()
    d1 = e1-s1
    print(node_size, d1)
    
    

1138499
1072
1138496 49.347442388534546
363
1138492 9.621734142303467
106
1138489 3.150555372238159
482709
1138485 5.187981605529785
644603
1138482 2.450512170791626
808


ValueError: list.remove(x): x not in list